In [40]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import log_loss, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

import sys
sys.path.append("../")
from src import cleaner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# Import data
data = pd.read_json('../data/data.json')

In [80]:
# Clean data and alter columns
cleaned_data = cleaner.clean_with_target(data)

cleaned_data['n_previous_payouts'] = cleaned_data['previous_payouts'].apply(lambda x: len(x))
cleaned_data.drop(columns='previous_payouts', inplace=True)

In [85]:
# Fill in na with 'None'
cleaned_data.isna().sum()
cleaned_data['country'] = cleaned_data['country'].fillna('None')
cleaned_data['delivery_method'] = cleaned_data['delivery_method'].fillna('None')
cleaned_data.isna().sum()

country                  0
currency                 0
delivery_method          0
email_domain             0
event_start              0
fb_published             0
listed                   0
object_id                0
payee_name               0
payout_type              0
user_age                 0
user_type                0
venue_country         1076
venue_latitude        1076
venue_longitude       1076
fraud                    0
n_previous_payouts       0
dtype: int64

In [95]:
(cleaned_data['country'] == 'None').sum()

81

# Set up Random Forest

In [55]:
df_dropped.columns

Index(['country', 'currency', 'delivery_method', 'email_domain', 'event_start',
       'fb_published', 'listed', 'object_id', 'payee_name', 'payout_type',
       'user_age', 'user_type', 'venue_country', 'venue_latitude',
       'venue_longitude', 'fraud', 'n_previous_payouts'],
      dtype='object')

In [86]:
# Limit columns for first round of modeling
to_keep = ['country', 'currency', 'delivery_method', 'email_domain', 'event_start',
       'fb_published', 'listed', 'payout_type', 'user_type', 'fraud', 'n_previous_payouts']
model_df = cleaned_data[to_keep]

316

In [70]:
y = model_df.pop('fraud')
X = model_df.copy()

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [72]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
#y_hat = model.predict(X_test)

ValueError: could not convert string to float: 'US'

In [73]:
model_df

,country,currency,delivery_method,email_domain,event_start,fb_published,listed,payout_type,user_type,n_previous_payouts
0,US,USD,0.0,gmail.com,1265594400,0,y,,1,0
1,US,USD,1.0,ruf.org,1296255600,0,n,CHECK,3,49
2,US,USD,1.0,pvsd.k12.ca.us,1295713800,0,y,CHECK,3,36
4,US,USD,0.0,artsandbusinesscouncil.org,1297440000,1,y,CHECK,3,49
5,US,USD,0.0,bluegrassstallions.com,1300053600,0,y,CHECK,1,5
...,...,...,...,...,...,...,...,...,...,...
14332,US,USD,0.0,yahoo.com,1361232000,0,n,,1,0
14333,US,USD,1.0,me.com,1365123600,0,y,CHECK,4,5
14334,,USD,0.0,yahoo.com,1368327600,1,y,ACH,4,1
14335,US,USD,0.0,velvetlist.com,1360890000,0,y,ACH,3,128
